In [1]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import joblib
import insightface

# 1. Inisialisasi InsightFace
model = insightface.app.FaceAnalysis(providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
model.prepare(ctx_id=0, det_size=(640, 640))

d:\miniconda3\envs\env_nugas\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'device_id': '0', 'has_user_compute_stream': '0', 'cudnn_conv1d_pad_to_nc1d': '0', 'user_compute_stream': '0', 'gpu_external_alloc': '0', 'gpu_mem_limit': '18446744073709551615', 'enable_cuda_graph': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'do_copy_in_default_stream': '1', 'cudnn_conv_use_max_workspace': '1', 'tunable_op_enable': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'prefer_nhwc': '0', 'use_ep_level_unified_stream': '0', 'use_tf32': '1', 'sdpa_kernel': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\LENOVO/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUD

In [2]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

datagen = ImageDataGenerator(
    rotation_range=25,             # random rotations (-25 to +25 degrees)
    width_shift_range=0.2,         # horizontal shift
    height_shift_range=0.2,        # vertical shift
    shear_range=0.2,               # shearing
    zoom_range=0.2,                # zoom in/out
    brightness_range=[0.6, 1.4],   # vary brightness
    channel_shift_range=30.0,      # randomly shift RGB values
    horizontal_flip=True,          # random left-right flip
    vertical_flip=True,            # (use only if upside-down faces make sense)
    fill_mode="nearest"            # fill strategy for empty pixels
)

input_root = "dataset"
output_root = "dataset_augmented"

os.makedirs(output_root, exist_ok=True)

# Loop over each class folder
for class_name in os.listdir(input_root):
    input_dir = os.path.join(input_root, class_name)
    output_dir = os.path.join(output_root, class_name)
    os.makedirs(output_dir, exist_ok=True)

    # Skip if not a folder
    if not os.path.isdir(input_dir):
        continue

    # Loop over each image in the class folder
    for filename in os.listdir(input_dir):
        img_path = os.path.join(input_dir, filename)
        if os.path.isdir(img_path):  # skip subfolders
            continue

        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        # Apply ALL augmentations one by one
        augmentations = [
            {"theta": 25},   # rotation
            {"tx": 0.2 * x.shape[1]},  # width shift
            {"ty": 0.2 * x.shape[2]},  # height shift
            {"shear": 20},   # shear
            {"zx": 1.2, "zy": 1.2},  # zoom
            {"brightness": 0.7},  # brightness (dark)
            {"brightness": 1.3},  # brightness (bright)
            {"channel_shift_intensity": 20},  # color shift
            {"flip_horizontal": True},  # horizontal flip
            {"flip_vertical": True},    # vertical flip
        ]

        for idx, params in enumerate(augmentations):
            transformed = datagen.apply_transform(x[0], params)
            aug_img = array_to_img(transformed)
            aug_img.save(os.path.join(output_dir, f"aug_{idx}_{filename}"))



In [3]:
dataset_path = 'dataset_augmented'  # ganti dengan path dataset Anda

In [4]:
# 2. Load dataset & generate embedding
X = []  # embeddings
y = []  # labels (nama orang)

In [5]:
for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    if not os.path.isdir(person_path):
        continue
    
    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        faces = model.get(img)
        if len(faces) > 0:
            emb = faces[0].embedding
            X.append(emb)
            y.append(person)

X = np.array(X)
y = np.array(y)


d:\miniconda3\envs\env_nugas\Lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [6]:
print(f"Total data: {len(X)} embeddings dari {len(set(y))} orang.")

Total data: 60 embeddings dari 3 orang.


In [7]:
print(f"Embedding dimension: {X.shape[1]}")

Embedding dimension: 512


In [8]:
from sklearn.model_selection import train_test_split

# Split data menjadi training dan validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # stratify untuk memastikan distribusi kelas tetap sama


In [9]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear', probability=True)

# Training pakai embedding
clf.fit(X_train, y_train)

# Evaluasi di validation
y_val_pred = clf.predict(X_val)
print("Validation accuracy:", accuracy_score(y_val, y_val_pred))


Validation accuracy: 1.0


In [10]:
import joblib
joblib.dump(clf, 'face_recognition_model.pkl')

['face_recognition_model.pkl']